In [13]:
import numpy as np 
import pandas as pd
import re
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from scipy import special
from tqdm import tqdm
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import SimpleImputer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, classification_report, accuracy_score, roc_curve
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.feature_selection import RFE
from boruta import BorutaPy

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20,.2f}'.format)

In [14]:
X_train = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/feature selection nbs/X_train_GA_final.csv')
y_train = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/y_train_large.csv').squeeze()
X_test = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/feature selection nbs/X_test_GA_final.csv')
y_test = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/y_test_large.csv').squeeze()

In [15]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from lightgbm import early_stopping, log_evaluation

#Assuming X_train, y_train, X_test, y_test are already defined

# Define parameter sets
lparams = {}

lparams[0] = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'num_leaves': 200,
    'max_bin': 500,
    'min_child_weight': 0.035,
    'subsample': 0.45,
    'colsample_bytree': 0.3,
    'min_data_in_leaf': 150,
    'max_depth': -1,
    'reg_alpha': 0.4,
    'reg_lambda': 0.7,
    'verbose': 1,
    'random_state': 0,  # Combining seed and bagging_seed for reproducibility
    'n_jobs': -1,
    'n_estimators': 30000
}

# Initialize models
model_0 = lgb.LGBMClassifier(**lparams[0])

In [16]:
# Fit models with early stopping
model_0.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='auc',
    callbacks=[
        early_stopping(stopping_rounds=100),
        log_evaluation(period=1)  
    ]
)

preds_0 = model_0.predict_proba(X_test)[:, 1]
auc_0 = roc_auc_score(y_test, preds_0)
print(f"Model 0 AUC: {auc_0:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036119 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 134729
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 319
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
[1]	valid_0's auc: 0.800225
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.822285
[3]	valid_0's

In [17]:
import lightgbm as lgb
import pandas as pd

# Assuming `lgb_model` is your trained LightGBM model
# `X_train` is your training data (features)

# Get feature importance by gain
importance_gain = model_0.booster_.feature_importance(importance_type='gain')
# Get feature names
feature_names = X_train.columns

# Create a DataFrame for feature importance
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance_gain': importance_gain
})

# Sort by importance
importance_df = importance_df.sort_values(by='importance_gain', ascending=False)

# Get top 50 features
top_50_model = importance_df.head(350)

# Display top 50 features by model's importance
top_50_model

,feature,importance_gain
4,d1_glucose_max,"273,025.24"
7,glucose_d1_max_icu_id_diff,"195,519.86"
22,glucose_apache,"126,566.04"
289,glucose_d1_max_profile_norm_rank,"68,344.60"
27,glucose_d1_mean,"65,663.44"
138,glucose_d1_h1_std,"54,435.75"
124,glucose_tot_change_value_range_normed,"46,327.42"
3,d1_hemaglobin_max_bmi_ratio,"37,896.69"
0,age,"34,599.96"
78,glucose_d1_max_apache_3j_diagnosis_norm_std,"32,889.74"


In [8]:
import shap

# Initialize SHAP explainer
explainer = shap.TreeExplainer(model_0)

# Calculate SHAP values for your dataset
shap_values = explainer.shap_values(X_train)

# Get mean absolute SHAP value for each feature
shap_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance_shap': np.mean(np.abs(shap_values), axis=0)
})

# Sort by SHAP importance
shap_importance = shap_importance.sort_values(by='importance_shap', ascending=False)

# Get top 50 features by SHAP importance
top_50_shap = shap_importance.head(50)

# Display top 50 features by SHAP values
print(top_50_shap)

In [ ]:
# Merge both importances for comparison
comparison_df = top_50_model.merge(top_50_shap, on='feature', how='outer').fillna(0)

# Display the comparison
print(comparison_df)

In [12]:
def feature_generation(df):

    df['weight_age'] = df['age'].values / df['weight'].values

    df['comorbidity_score'] = df['aids'].values * 23 + df['cirrhosis'].values * 4 + \
                                 df['hepatic_failure'].values * 16 + df['immunosuppression'].values * 10 + \
                              df['leukemia'].values * 10 + df['lymphoma'].values * 13 + df['solid_tumor_with_metastasis'].values * 11
    

    lab_col = [c for c in df.columns if ((c.startswith("h1")) | (c.startswith("d1")))]
    lab_col_names = list(set(list(map(lambda i: i[3: -4], lab_col))))

    first_h = []
    for v in tqdm(lab_col_names):
        colsx = [x for x in df.columns if v in x]
        df[v + "_d1_h1_max_eq"] = (df[f"d1_{v}_max"] == df[f"h1_{v}_max"]).astype(np.int8)


        for freq in ['h1', 'd1']:
            df[v + f"_{freq}_value_range"] = df[f"{freq}_{v}_max"].subtract(df[f"{freq}_{v}_min"])
            df[v + f"_{freq}_mean"] =np.nanmean(df[[f"{freq}_{v}_max", f"{freq}_{v}_min"]].values, axis=1)
            df[v + f"_{freq}_std"] = np.nanstd(df[[f"{freq}_{v}_max", f"{freq}_{v}_min"]].values, axis=1)

            for g in ['apache_3j_diagnosis', 'profile', 'icu_id']:
                for m in ['max', 'min']:
                    temp = df[[g, f"{freq}_{v}_{m}"]].groupby(g)
                    df[v + f"_{freq}_{m}_{g}_mean"] = temp.transform('mean')
                    df[v + f"_{freq}_{m}_{g}_diff"] = df[v + f"_{freq}_{m}_{g}_mean"].subtract(df[f"{freq}_{v}_{m}"])
                    df[v + f"_{freq}_{m}_{g}_std"] = temp.transform('std')
                    df[v + f"_{freq}_{m}_{g}_norm_std"] = df[v + f"_{freq}_{m}_{g}_std"].div(df[f"{freq}_{v}_{m}"])
                    df[v + f"_{freq}_{m}_{g}_rank"] =temp.transform('rank')
                    df[v + f"_{freq}_{m}_{g}_norm_rank"] = df[v + f"_{freq}_{m}_{g}_rank"].div(df[v + f"_{freq}_{m}_{g}_count"])
                    df[v + f"_{freq}_{m}_{g}_skew"] = temp.transform('skew')

            if v + "_apache" in colsx:
                for m in ['max', 'min']:
                    df[v + f"_apache_{freq}_{m}_ratio"] = df[f"{freq}_{v}_{m}"].div(df[v + "_apache"])

            for m in ['max', 'min']:
                df[f"{freq}_{v}_{m}_bmi_ratio"] = df[f"{freq}_{v}_{m}"].div(df['bmi'])
                df[f"{freq}_{v}_{m}_weight_ratio"] = df[f"{freq}_{v}_{m}"].div(df['weight'])

        df[v + "_range_between_d_h"] = df[v + "_d1_mean"].values - df[v + "_h1_mean"].values
        df[v + "_d1_h1_mean"] = np.nanmean(df[[f"d1_{v}_max", f"d1_{v}_min", f"h1_{v}_max", f"h1_{v}_min"]].values, axis=1)
        df[v + "_d1_h1_std"] = np.nanstd(df[[f"d1_{v}_max", f"d1_{v}_min", f"h1_{v}_max", f"h1_{v}_min"]].values, axis=1)

        df[v + "_tot_change_value_range_normed"] = abs((df[v + "_d1_value_range"].div(df[v + "_h1_value_range"])))

    df['h1_diasbp_invnoninv_min_diff'] = df['h1_diasbp_invasive_min'].div(df['h1_diasbp_noninvasive_min'])
    df["d1_lactate_min_div_diasbp_min"] = df["d1_lactate_min"].div(df["d1_diasbp_min"])



    return df

In [ ]:
def feature_generation(df):

    agg = df['icu_id'].value_counts().to_dict()
    df['icu_id_counts'] = np.log1p(df['icu_id'].map(agg))

    df['weight_age'] = df['age'].values / df['weight'].values

    df['comorbidity_score'] = df['aids'].values * 23 + df['cirrhosis'].values * 4 + \
                                 df['hepatic_failure'].values * 16 + df['immunosuppression'].values * 10 + \
                              df['leukemia'].values * 10 + df['lymphoma'].values * 13 + df['solid_tumor_with_metastasis'].values * 11
    

    df['bmi_type'] = df.bmi.fillna(0).apply(lambda x: 5 * (round(int(x) / 5)))
    df['age_type'] = df.age.fillna(0).apply(lambda x: 10 * (round(int(x) / 10)))
    df['gcs_sum_type'] = df.gcs_sum.fillna(0).apply(lambda x: 2.5 * (round(int(x) / 2.5))).divide(2.5)

    IDENTIFYING_COLS = ['age_type', 'gcs_sum_type', 'ethnicity', 'gender', 'bmi_type']
    df['profile'] = df[IDENTIFYING_COLS].apply(lambda x: hash(tuple(x)), axis=1)

    lab_col = [c for c in df.columns if ((c.startswith("h1")) | (c.startswith("d1")))]
    lab_col_names = list(set(list(map(lambda i: i[3: -4], lab_col))))

    first_h = []
    for v in tqdm(lab_col_names):
        colsx = [x for x in df.columns if v in x]
        df[v + "_nans"] = df.loc[:, colsx].isna().sum(axis=1)
        df[v + "_d1_h1_max_eq"] = (df[f"d1_{v}_max"] == df[f"h1_{v}_max"]).astype(np.int8)
        df[v + "_d1_h1_min_eq"] = (df[f"d1_{v}_min"] == df[f"h1_{v}_min"]).astype(np.int8)


        for freq in ['h1', 'd1']:
            df[v + f"_{freq}_value_range"] = df[f"{freq}_{v}_max"].subtract(df[f"{freq}_{v}_min"])
            df[v + f"_{freq}_zero_range"] = (df[v + f"_{freq}_value_range"] == 0).astype(np.int8)
            df[v + f"_{freq}_mean"] =np.nanmean(df[[f"{freq}_{v}_max", f"{freq}_{v}_min"]].values, axis=1)
            df[v + f"_{freq}_std"] = np.nanstd(df[[f"{freq}_{v}_max", f"{freq}_{v}_min"]].values, axis=1)

            for g in ['apache_3j_diagnosis', 'profile', 'icu_id']:
                for m in ['max', 'min']:
                    temp = df[[g, f"{freq}_{v}_{m}"]].groupby(g)
                    df[v + f"_{freq}_{m}_{g}_mean"] = temp.transform('mean')
                    df[v + f"_{freq}_{m}_{g}_diff"] = df[v + f"_{freq}_{m}_{g}_mean"].subtract(df[f"{freq}_{v}_{m}"])
                    df[v + f"_{freq}_{m}_{g}_std"] = temp.transform('std')
                    df[v + f"_{freq}_{m}_{g}_norm_std"] = df[v + f"_{freq}_{m}_{g}_std"].div(df[f"{freq}_{v}_{m}"])
                    df[v + f"_{freq}_{m}_{g}_rank"] =temp.transform('rank')
                    df[v + f"_{freq}_{m}_{g}_count"] = temp.transform('count')
                    df[v + f"_{freq}_{m}_{g}_norm_rank"] = df[v + f"_{freq}_{m}_{g}_rank"].div(df[v + f"_{freq}_{m}_{g}_count"])
                    df[v + f"_{freq}_{m}_{g}_skew"] = temp.transform('skew')

            if v + "_apache" in colsx:
                for m in ['max', 'min']:
                    df[v + f"_apache_{freq}_{m}_ratio"] = df[f"{freq}_{v}_{m}"].div(df[v + "_apache"])

            for m in ['max', 'min']:
                df[f"{freq}_{v}_{m}_bmi_ratio"] = df[f"{freq}_{v}_{m}"].div(df['bmi'])
                df[f"{freq}_{v}_{m}_weight_ratio"] = df[f"{freq}_{v}_{m}"].div(df['weight'])

        df[v + "_range_between_d_h"] = df[v + "_d1_mean"].values - df[v + "_h1_mean"].values
        df[v + "_d1_h1_mean"] = np.nanmean(df[[f"d1_{v}_max", f"d1_{v}_min", f"h1_{v}_max", f"h1_{v}_min"]].values, axis=1)
        df[v + "_d1_h1_std"] = np.nanstd(df[[f"d1_{v}_max", f"d1_{v}_min", f"h1_{v}_max", f"h1_{v}_min"]].values, axis=1)



    return df

In [ ]:
def feature_generation(df):


    IDENTIFYING_COLS = ['age_type', 'gcs_sum_type', 'ethnicity', 'gender', 'bmi_type']
    df['profile'] = df[IDENTIFYING_COLS].apply(lambda x: hash(tuple(x)), axis=1)

    lab_col = [c for c in df.columns if ((c.startswith("h1")) | (c.startswith("d1")))]
    lab_col_names = list(set(list(map(lambda i: i[3: -4], lab_col))))

    first_h = []
    for v in tqdm(lab_col_names):
        colsx = [x for x in df.columns if v in x]
        df[v + "_nans"] = df.loc[:, colsx].isna().sum(axis=1)
        df[v + "_d1_h1_max_eq"] = (df[f"d1_{v}_max"] == df[f"h1_{v}_max"]).astype(np.int8)
        df[v + "_d1_h1_min_eq"] = (df[f"d1_{v}_min"] == df[f"h1_{v}_min"]).astype(np.int8)


        for freq in ['h1', 'd1']:
            df[v + f"_{freq}_value_range"] = df[f"{freq}_{v}_max"].subtract(df[f"{freq}_{v}_min"])
            df[v + f"_{freq}_zero_range"] = (df[v + f"_{freq}_value_range"] == 0).astype(np.int8)
            df[v + f"_{freq}_mean"] =np.nanmean(df[[f"{freq}_{v}_max", f"{freq}_{v}_min"]].values, axis=1)
            df[v + f"_{freq}_std"] = np.nanstd(df[[f"{freq}_{v}_max", f"{freq}_{v}_min"]].values, axis=1)

            for g in ['apache_3j_diagnosis', 'profile', 'icu_id']:
                for m in ['max', 'min']:
                    temp = df[[g, f"{freq}_{v}_{m}"]].groupby(g)
                    df[v + f"_{freq}_{m}_{g}_mean"] = temp.transform('mean')
                    df[v + f"_{freq}_{m}_{g}_diff"] = df[v + f"_{freq}_{m}_{g}_mean"].subtract(df[f"{freq}_{v}_{m}"])
                    df[v + f"_{freq}_{m}_{g}_std"] = temp.transform('std')
                    df[v + f"_{freq}_{m}_{g}_norm_std"] = df[v + f"_{freq}_{m}_{g}_std"].div(df[f"{freq}_{v}_{m}"])
                    df[v + f"_{freq}_{m}_{g}_rank"] =temp.transform('rank')
                    df[v + f"_{freq}_{m}_{g}_count"] = temp.transform('count')
                    df[v + f"_{freq}_{m}_{g}_norm_rank"] = df[v + f"_{freq}_{m}_{g}_rank"].div(df[v + f"_{freq}_{m}_{g}_count"])
                    df[v + f"_{freq}_{m}_{g}_skew"] = temp.transform('skew')

            if v + "_apache" in colsx:
                for m in ['max', 'min']:
                    df[v + f"_apache_{freq}_{m}_ratio"] = df[f"{freq}_{v}_{m}"].div(df[v + "_apache"])

            for m in ['max', 'min']:
                df[f"{freq}_{v}_{m}_bmi_ratio"] = df[f"{freq}_{v}_{m}"].div(df['bmi'])
                df[f"{freq}_{v}_{m}_weight_ratio"] = df[f"{freq}_{v}_{m}"].div(df['weight'])

        df[v + "_range_between_d_h"] = df[v + "_d1_mean"].values - df[v + "_h1_mean"].values
        df[v + "_d1_h1_mean"] = np.nanmean(df[[f"d1_{v}_max", f"d1_{v}_min", f"h1_{v}_max", f"h1_{v}_min"]].values, axis=1)
        df[v + "_d1_h1_std"] = np.nanstd(df[[f"d1_{v}_max", f"d1_{v}_min", f"h1_{v}_max", f"h1_{v}_min"]].values, axis=1)



    return df